In [2]:
%pip install torch torchvision
%pip install ninja click requests tqdm pyspng imageio-ffmpeg==0.4.3
%pip install matplotlib


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 972.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.4/253.4 kB 4.4 MB/s eta 0:00:00a 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 2.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 2.4 MB/s eta 0:00:0000:01

[notice] A new rel

In [6]:
import os, glob, random
from PIL import Image

SRC         = "brownPhotos"            # expects: brownPhotos/psoriasis, brownPhotos/eczema
DST         = "brownPhotos_256sq"      # output root
CLASSES     = ["psoriasis", "eczema"]
EXTS        = ("*.jpg","*.jpeg","*.png","*.JPG","*.JPEG","*.PNG")
TARGET      = (256, 256)
TRAIN_RATIO = 0.8
SEED        = 1337

random.seed(SEED)

def list_images(folder, patterns):
    files = []
    for pat in patterns:
        files.extend(glob.glob(os.path.join(folder, pat)))
        files.extend(glob.glob(os.path.join(folder, "**", pat), recursive=True))  # include subfolders if any
    return sorted(files)

def save_square_resized(in_path, out_path, size):
    img = Image.open(in_path).convert("RGB")
    w, h = img.size
    m = min(w, h)
    # center-crop
    left = (w - m) // 2
    top  = (h - m) // 2
    img  = img.crop((left, top, left + m, top + m))
    # resize & save
    img = img.resize(size, Image.LANCZOS)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    img.save(out_path, quality=95)

# ensure output dirs exist
for split in ("train", "test"):
    for cls in CLASSES:
        os.makedirs(os.path.join(DST, split, cls), exist_ok=True)

for cls in CLASSES:
    src_folder = os.path.join(SRC, cls)  # <— NOTE: no 'train/test' here
    files = list_images(src_folder, EXTS)
    print(f"{cls}: found {len(files)} images in '{src_folder}'")
    if not files:
        continue

    random.shuffle(files)
    k = int(len(files) * TRAIN_RATIO)
    split_map = {"train": files[:k], "test": files[k:]}

    for split, paths in split_map.items():
        dst_folder = os.path.join(DST, split, cls)
        saved = 0
        for path in paths:
            fn = os.path.basename(path)
            outpath = os.path.join(dst_folder, fn)
            try:
                save_square_resized(path, outpath, TARGET)
                saved += 1
            except Exception as e:
                print(f"  ! Skipped {fn}: {e}")
        print(f"{split}/{cls}: saved {saved} images to {dst_folder}")

print("Done preprocessing!")

psoriasis: found 220 images in 'brownPhotos/psoriasis'
train/psoriasis: saved 176 images to brownPhotos_256sq/train/psoriasis
test/psoriasis: saved 44 images to brownPhotos_256sq/test/psoriasis
eczema: found 42 images in 'brownPhotos/eczema'
train/eczema: saved 33 images to brownPhotos_256sq/train/eczema
test/eczema: saved 9 images to brownPhotos_256sq/test/eczema
Done preprocessing!


In [ ]:
from pathlib import Path

root = Path("dataset")
splits = ["train", "test"]
classes = ["eczema", "psoriasis"]
exts = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".gif"}

def count_images(folder: Path) -> int:
    return sum(1 for f in folder.rglob("*") if f.suffix.lower() in exts)

total = 0
for split in splits:
    split_total = 0
    for cls in classes:
        p = root / split / cls
        n = count_images(p)
        split_total += n
        total += n
        print(f"{p}: {n}")
    print(f"{root/split} TOTAL: {split_total}\n")
print(f"GRAND TOTAL: {total}")


cwd: /Users/sophiasalta/Desktop/admu/thesis/derment_dataset
Python: /usr/local/bin/python3
SRC exists?  True
eczema exists?  True
psoriasis exists?  True
eczema sample files: ['brownPhotos/eczema/atopicEczema-1.jpg', 'brownPhotos/eczema/atopicEczema-10.jpg', 'brownPhotos/eczema/atopicEczema-12.jpg', 'brownPhotos/eczema/atopicEczema-2.jpg', 'brownPhotos/eczema/atopicEczema-3.jpg']
